# `gmxapi` Practice
In this notebook, our goal is to use `gmxapi` to run a standard MD simulation, an expanded ensemble simulaiton, and a 1D alchemical metadynamics simulation. Note that before running this notebook, one should first load in GROMACS and activate corresponding virtual environment as needed. Delete the folders like `standard_MD`, `EE`, `lambda_MetaD`, and `gmxapi.commadline.cli*_i0` if they exist. These folders will be generated later along the way. Notably, the examples set up in this notebooks are demonstrating the usage of `gmxapi` with MPI-enabled GROMACS. Readers are recommended to first look at the gmxapi tutorial for GROMCS running with thread-MPI. Here we are using an interactive node with 128 cores on Bridgees-2.

In [1]:
import sys
sys.path.append('/jet/home/wehs7661/gmxapi_21.4/lib/python3.8/site-packages')  # might not be necessary; change the path as needed
import gmxapi as gmx

As a record, here we print out the version of gmxapi used:

In [2]:
gmx.__version__

'0.2.3'

## Section 1. Running a standard MD simulation using `gmxapi`

To get started, we set up the directories as below:

In [3]:
%%bash
mkdir standard_MD && cd standard_MD
cp -r ../../inputs/sys2_init.gro .
cp -r ../../inputs/sys2.top .
mkdir Box Sol EM Equil MD Topology
mv *top Topology/.
cd EM && cp ../../../inputs/mdp_files/em.mdp .
cd ../Equil && mkdir NVT && cd NVT && cp ../../../../inputs/mdp_files/nvt_equil.mdp .
cd ../ && mkdir NPT && cd NPT && cp ../../../../inputs/mdp_files/npt_equil.mdp .
cd ../../MD && cp ../../../inputs/mdp_files/md.mdp .

### 1. Construct the simulation box
Here we use `sys2_init.gro` as the input file, which is the same as `simple_system2.gro` except that the box vector was removed. Normally, the GROMACS command would be as follows if the working directory is the path of this notebook:
```
gmx_mpi editconf -f standard_MD/sys2_init.gro -o standard_MD/Box/sys2_box.gro -bt dodecahedron -d 1 -c
```
Using `gmxapi`, we first create an object using `gmx.commandline_operation` and run `object.run()`. And since we need to call `mpirun` externally, we write `box.py` instead of executing the command internally from Python. 

In [4]:
%%bash
cd standard_MD/Box/
code="import sys 
# sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')  # actually not needed
import gmxapi as gmx

box = gmx.commandline_operation('gmx_mpi',
                               arguments=['editconf', '-bt', 'dodecahedron', '-d', '1', '-c'],
                               input_files={'-f': '../sys2_init.gro'},
                               output_files={'-o': 'sys2_box.gro'})
box.run()
print(f'Return code of the process: {box.output.returncode.result()}\n')
if box.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {box.output.erroroutput.result()}')
"
echo "${code}" >> box.py

mpirun -np 1 python -m mpi4py box.py

Return code of the process: 0



Unlike the case presented in the tutorial of gmxapi with GROMACS running with thread-MPI, no folders like `gmxapi.commandline.cli*_i0` is created and the working directory of the code is just where the code is. If `output_files` is not specified, the filename of the output GRO file will just be the default file name (which is `out.gro` in this case) and it will still be saved in the folder `Box`. Also, here the object does not have attributes such as `stderr` and `stdout`, but we can still check `returncode` and `erroroutput` as needed. 

### Question to address: How to print out stderr and stdout?

### 2. Solvate the system

Again, to solvate the system, the original GROMACS command is as follows:
```
gmx_mpi solvate -cp standard_MD/Box/sys2_box.gro -p standard_MD/Topology/sys2.top -o sys2_sol.gro -cs
```
Note that since we need to execute commands in a separate Python script, we can't retrieve the output filename from an object like `box.output`. (Or we can choose to execute all GROMACS commands in a Python script switching different working directories.) Below we write a Python script `solvate.py` in the folder `Sol`.

In [5]:
%%bash
cd standard_MD/Sol/
code="import sys 
# sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')  # actually not needed
import gmxapi as gmx

solvate = gmx.commandline_operation('gmx_mpi',
                                   arguments=['solvate', '-cs'],
                                   input_files={
                                       '-cp': '../Box/sys2_box.gro',
                                       '-p': '../Topology/sys2.top'},
                                   output_files={'-o': 'sys2_sol.gro'})
solvate.run()
print(f'Return code of the process: {solvate.output.returncode.result()}\n')
if solvate.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {solvate.output.erroroutput.result()}')
"
echo "${code}" >> solvate.py

mpirun -np 1 python -m mpi4py solvate.py

Return code of the process: 0



### 3. Energy minimization

Again, we run the following GROMACS commands using `gmxapi`:
```
gmx_mpi grompp -f standard_MD/EM/em.mdp -c standard_MD/Sol/sys2_sol.gro -p standard_MD/Topology/sys2.top -o standard_MD/EM/sys2_em.tpr
gmx_mpi mdrun -s standard_MD/EM/sys2_em.tpr -c standard_MD/EM/sys2_em.gro -g standard_MD/EM/em.log -e standard_MD/EM/em.edr
```
Notably, with MPI-enabled, the first command only requires 1 core to run, while the second often requires multiple. On Bridges-2, we typically use `-np 64` or `-np 128`. Therefore, we need to separate Python scripts for these two commands such that we can execute them with different numbers of processors. 

In [6]:
%%bash
cd standard_MD/EM/
code="import sys 
# sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')  # actually not needed
import gmxapi as gmx

grompp_em = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': 'em.mdp',
                                         '-c': '../Sol/sys2_sol.gro',
                                         '-p': '../Topology/sys2.top'},
                                     output_files={'-o': 'sys2_em.tpr'})
grompp_em.run()


print(f'Return code of the process: {grompp_em.output.returncode.result()}\n')
if grompp_em.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {grompp_em.output.erroroutput.result()}')
"
echo "${code}" >> grompp_em.py



mpirun -np 1 python -m mpi4py grompp_em.py


Return code of the process: 0



In [7]:
%%bash
cd standard_MD/EM/

code="import sys 
# sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')  # actually not needed
import gmxapi as gmx

em = gmx.commandline_operation('gmx_mpi',
                              arguments=['mdrun'],
                              input_files={'-s': 'sys2_em.tpr'},
                              output_files={
                                  '-c': 'sys2_em.gro',
                                  '-e': 'em.edr',
                                  '-g': 'em.log',
                                  '-o': 'em.trr'})
em.run()
print(f'Return code of the process: {em.output.returncode.result()}\n')
if em.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {em.output.erroroutput.result()}')
"
echo "${code}" >> em.py

mpirun -np 64 python -m mpi4py em.py

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return c

### 4. Equilibration

To run NVT and then NPT equilibration using GROMACS, we use the following commands:
```
gmx_mpi grompp -f standard_MD/Equil/NVT/nvt_equil.mdp -c standard_MD/EM/sys2_em.gro -p standard_MD/Topology/sys2.top -o standard_MD/Equil/NVT/sys2_equil.tpr
gmx_mpi mdrun -s standard_MD/Equil/NVT/sys2_equil.tpr -c standard_MD/Equil/NVT/sys2_equil.gro -g standard_MD/Equil/NVT/equil.log -e standard_MD/Equil/NVT/equil.edr
gmx_mpi grompp -f npt_equil.mdp -c ../NVT/sys2_equil.gro -t ../NVT/state.cpt -p ../../Topology/sys2.top -o sys2_equil.tpr
gmx_mpi mdrun -s sys2_equil.tpr -c sys2_equil.gro -g equil.log -e equil.edr
```

In [8]:
%%bash
cd standard_MD/Equil/NVT/

code="import sys 
import gmxapi as gmx

grompp_nvt = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': 'nvt_equil.mdp',
                                         '-c': '../../EM/sys2_em.gro',
                                         '-p': '../../Topology/sys2.top'},
                                     output_files={'-o': 'sys2_equil.tpr'})
grompp_nvt.run()
print(f'Return code of the process: {grompp_nvt.output.returncode.result()}\n')
if grompp_nvt.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {grompp_nvt.output.erroroutput.result()}')
"
echo "${code}" >> grompp_nvt.py

mpirun -np 1 python -m mpi4py grompp_nvt.py

Return code of the process: 0



In [9]:
%%bash
cd standard_MD/Equil/NVT/

code="import sys 
import gmxapi as gmx

nvt = gmx.commandline_operation('gmx_mpi',
                               arguments=['mdrun'],
                               input_files={'-s': 'sys2_equil.tpr'},
                               output_files={
                                   '-c': 'sys2_equil.gro',
                                   '-e': 'equil.edr',
                                   '-g': 'equil.log',
                                   '-o': 'equil.trr',
                                   '-cpo': 'state.cpt'})
nvt.run()
print(f'Return code of the process: {nvt.output.returncode.result()}\n')
if nvt.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {nvt.output.erroroutput.result()}')
"
echo "${code}" >> nvt.py

mpirun -np 16 python -m mpi4py nvt.py

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0



In [10]:
%%bash
cd standard_MD/Equil/NPT/

code="import sys 
import gmxapi as gmx

grompp_npt = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': 'npt_equil.mdp',
                                         '-c': '../NVT/sys2_equil.gro',
                                         '-p': '../../Topology/sys2.top',
                                         '-t': '../NVT/state.cpt'},
                                     output_files={'-o': 'sys2_equil.tpr'})
grompp_npt.run()
print(f'Return code of the process: {grompp_npt.output.returncode.result()}\n')
if grompp_npt.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {grompp_npt.output.erroroutput.result()}')
"
echo "${code}" >> grompp_npt.py

mpirun -np 1 python -m mpi4py grompp_npt.py

Return code of the process: 0



In [11]:
%%bash
cd standard_MD/Equil/NPT/

code="import sys 
import gmxapi as gmx

npt = gmx.commandline_operation('gmx_mpi',
                               arguments=['mdrun'],
                               input_files={'-s': 'sys2_equil.tpr'},
                               output_files={
                                   '-c': 'sys2_equil.gro',
                                   '-e': 'equil.edr',
                                   '-g': 'equil.log',
                                   '-o': 'equil.trr',
                                   '-cpo': 'state.cpt'})
npt.run()
print(f'Return code of the process: {npt.output.returncode.result()}\n')
if npt.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {npt.output.erroroutput.result()}')
"
echo "${code}" >> npt.py

mpirun -np 16 python -m mpi4py npt.py

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0



### 5. Launch the simulation

Lastly, we use the `gmxapi` to run the following commands:
```
gmx_mpi grompp -f standard_MD/MD/md.mdp -c standard_MD/Equil/NPT/sys2_equil.gro -p ../Topology/sys2.top -o sys2_md.tpr -t ../Equil/NPT/state.cpt
gmx_mpi mdrun -s sys2_md.tpr -x sys2_md.xtc -c sys2_md.gro -g md.log -e md.edr
```

In [12]:
%%bash
cd standard_MD/MD/

code="import sys 
import gmxapi as gmx

grompp_md = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': 'md.mdp',
                                         '-c': '../Equil/NPT/sys2_equil.gro',
                                         '-p': '../Topology/sys2.top',
                                         '-t': '../Equil/NPT/state.cpt'},
                                     output_files={'-o': 'sys2_md.tpr'})
grompp_md.run()
print(f'Return code of the process: {grompp_md.output.returncode.result()}\n')
if grompp_md.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {grompp_md.output.erroroutput.result()}')
"
echo "${code}" >> grompp_md.py

mpirun -np 1 python -m mpi4py grompp_md.py

Return code of the process: 0



In [13]:
%%bash
cd standard_MD/MD

code="import sys 
import gmxapi as gmx

md = gmx.commandline_operation('gmx_mpi',
                              arguments=['mdrun'],
                              input_files={'-s': 'sys2_md.tpr'},
                              output_files={
                                  '-c': 'sys2_md.gro',
                                  '-e': 'md.edr',
                                  '-g': 'md.log',
                                  '-o': 'md.trr'})
md.run()
print(f'Return code of the process: {md.output.returncode.result()}\n')
if md.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {md.output.erroroutput.result()}')
"
echo "${code}" >> md.py

mpirun -np 16 python -m mpi4py md.py

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0



## Section 2. Running expanded ensemble simulations using `gmxapi`

Here we take the output of the standard MD done in Section 1 as the input of expanded ensemble. The way we run exapnded ensemble simulations using `gmxapi` is pretty much the same as what we've done in Section 1. Here is just a section showing that `gmxapi` can also run advanced sampling methods.

To make the demonstration a little more interesting, instead of running just 1 expanded ensemble, the goal of this section is to run 16 expanded ensemble at the same time to show `gmxapi`'s ability to run an emsemble of simulation. Note that in this case, the 16 simulations are independent of each other and this can only be done in `gmx.mdrun`, not `gmx.commandline_operation`.

Again, here we first set up relevant directory strucutres.

In [3]:
%%bash 
mkdir EE && cd EE 
mkdir case_1 case_2
cp ../../inputs/mdp_files/expanded.mdp .
cp ../standard_MD/Topology/*top .

Then, we use `grompp` to generate the input `tpr` file.

In [4]:
%%bash
cd EE/

code="import sys 
import gmxapi as gmx


grompp_EE = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': 'expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': 'sys2.top'},
                                     output_files={'-o': f'sys2_EE.tpr'})
grompp_EE.run()
print(f'Return code of the process: {grompp_EE.output.returncode.result()}\n')
if grompp_EE.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {grompp_EE.output.erroroutput.result()}')
"
echo "${code}" >> grompp_EE.py

mpirun -np 1 python -m mpi4py grompp_EE.py

Return code of the process: 0



### 1. Running 1 expanded ensemble simulation using `gmxapi`

Before we get to running 16 simulations using `gmxapi`, let's first try running only 1 expanded ensemble simulation using `gmxapi`.

In [5]:
%%bash 
cd EE/case_1

code="import sys 
import gmxapi as gmx

# tpr_list = ['../sys2_EE.tpr' for i in range(16)]  # the tpr files are the same
EE_inputs = gmx.read_tpr('../sys2_EE.tpr')
EE = gmx.mdrun(EE_inputs)
EE.run()"
    
echo "${code}" >> EE.py

mpirun -np 16 python -m mpi4py EE.py 

--------------------------------------------------------------------------
A process has executed an operation involving a call to the
"fork()" system call to create a child process.  Open MPI is currently
operating in a condition that could result in memory corruption or
other system errors; your job may hang, crash, or produce silent
data corruption.  The use of fork() (or system() or other calls that
create child processes) is strongly discouraged.

The process that invoked fork was:

  Local host:          [[31532,1],8] (PID 86801)

If you are *absolutely sure* that your application will successfully
and correctly survive a call to fork(), you may disable this warning
by setting the mpi_warn_on_fork MCA parameter to 0.
--------------------------------------------------------------------------
Reading file ../sys2_EE.tpr, VERSION 2021.4-plumed-2.8.0 (single precision)
Reading file ../sys2_EE.tpr, VERSION 2021.4-plumed-2.8.0 (single precision)
Reading file /ocean/projects/cts160011p/

Process is interrupted.


In [38]:
%%bash
cd EE/case_1

code="import sys 
import gmxapi as gmx

EE = gmx.commandline_operation('gmx_mpi',
                              arguments=['mdrun'],
                              input_files={'-s': '../sys2_EE.tpr'},
                              output_files={
                                  '-c': '../sys2_EE.gro',
                                  '-e': 'EE.edr',
                                  '-g': 'EE.log',
                                  '-o': 'EE.trr'})
EE.run()
print(f'Return code of the process: {EE.output.returncode.result()}\n')
if EE.output.returncode.result() != 0:
    print(f'Error of the process:\n\n {EE.output.erroroutput.result()}')
"
echo "${code}" >> EE.py

mpirun -np 16 python -m mpi4py EE.py

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0

Return code of the process: 0



### 2. Running 16 expanded ensemble simulations using `gmxapi`

In [57]:
%%bash 
cd EE/case_2

code="import sys 
import gmxapi as gmx

tpr_list = ['../sys2_EE.tpr' for i in range(16)]  # the tpr files are the same
EE_inputs = gmx.read_tpr(tpr_list)
EE = gmx.mdrun(EE_inputs, runtime_args={'ntomp': 1})
EE.run()"
    
echo "${code}" >> EE.py

mpirun -np 16 python -m mpi4py EE.py 

Traceback (most recent call last):
  File "/opt/packages/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/packages/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/jet/home/wehs7661/gmxapi_21.4/lib/python3.8/site-packages/mpi4py/__main__.py", line 7, in <module>
    main()
  File "/jet/home/wehs7661/gmxapi_21.4/lib/python3.8/site-packages/mpi4py/run.py", line 198, in main
    run_command_line(args)
  File "/jet/home/wehs7661/gmxapi_21.4/lib/python3.8/site-packages/mpi4py/run.py", line 47, in run_command_line
    run_path(sys.argv[0], run_name='__main__')
  File "/opt/packages/anaconda3/lib/python3.8/runpy.py", line 265, in run_path
    return _run_module_code(code, init_globals, run_name,
  File "/opt/packages/anaconda3/lib/python3.8/runpy.py", line 97, in _run_module_code
    _run_code(code, mod_globals, init_globals,
  File "/opt/packages/anaconda3/lib/python3.8

CalledProcessError: Command 'b'cd EE/case_2\n\ncode="import sys \nimport gmxapi as gmx\n\ntpr_list = [\'../sys2_EE.tpr\' for i in range(16)]  # the tpr files are the same\nEE_inputs = gmx.read_tpr(tpr_list)\nEE = gmx.mdrun(EE_inputs, runtime_args={\'ntomp\': 1})\nEE.run()"\n    \necho "${code}" >> EE.py\n\nmpirun -np 16 python -m mpi4py EE.py \n'' returned non-zero exit status 1.

Notably, the value of `n` should at least be 4 since we have 4 simulations to run. Setting the value of `n` as 4, the performance of each simulation is around 15 ns/day, while the performance of 1 expanded ensemble simulation (using all the 16 cores) was around 240 ns/day. The parallelizing simulations here are not ideal because we ran all the simulations within 1 node. If multiple nodes are available, the scaling should be better. (In our case here, simulations with `-n` being 16 would be even slower.)

## Section 3. Running alchemical metadynamics simulations using `gmxapi`

Lastly, we want to see if `gmxapi` works with PLUMED, or more specifically, if `gmxapi` can run alchemical metadynamics simulations, including 1D and 2D cases. Note that here we are using a certain commit of PLUMED 2.7.3, which is the commit where the issue of alchemical metadynamics had been fixed but had not merged to the branch `v2.8` and changed the way that the mdp file should have been specified. Therefore, in this specific version of PLUMED, a mdp file for alchemical metadynamics would just be the same as the one used in expanded ensemble (weight-equilibrating phase). 

### 1. 1D alchemical metadynamics
Again, we first set up directory structures for a 1D alchemical metadynamics simulation. 

In [58]:
%%bash 
mkdir lambda_MetaD && cd lambda_MetaD 
mkdir 1D_case 2D_case && cd 1D_case
cp ../../../inputs/plumed_1.dat . 
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [20]:
grompp_lambda_1 = gmx.commandline_operation('gmx_mpi', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../lambda_MetaD/1D_case/expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': '../lambda_MetaD/1D_case/sys2.top'},
                                     output_files={'-o': '../lambda_MetaD/1D_case/sys2_lambda.tpr'})
grompp_lambda_1.run()
gmx_output(grompp_lambda_1)

The process was executed successfully.


Similarly, we run the simulation by executing a Python script from the terminal. Note that the Python script `lambda_MetaD.py` was executed in the folder `lambda_MetaD`, so the paths of the input and output files in the script below should be changed accordingely.

In [21]:
%%bash 
cd lambda_MetaD/1D_case 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

lambda_MetaD = gmx.commandline_operation('gmx',
                              arguments=['mdrun', '-plumed', '../plumed_1.dat'],
                              input_files={'-s': '../sys2_lambda.tpr'},
                              output_files={
                                  '-c': '../sys2_lambda.gro',
                                  '-e': '../lambda.edr',
                                  '-g': '../lambda.log',
                                  '-o': '../lambda.trr'})
lambda_MetaD.run()

if lambda_MetaD.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {lambda_MetaD.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')"

echo "${code}" >> lambda_MetaD.py

python lambda_MetaD.py

The process was executed successfully.


Nice! So `gmxapi` did work with PLUMED! (It's kind of weird that the bottom of the log file was not printed here though.) Note that since the path of the PLUMED output files can not be specified from the GROMACS command but defined in `plumed.dat`, all the PLUMED outputs will be saved in the folder `gmxapi.commandline.cli0_i0 `.

### 2. 2D alchemical metadynamics

We repeat the protocol above but with a different PLUMED input file. Here we just want to make sure that `gmxapi` also works with 2D cases. 

In [22]:
%%bash 
cd lambda_MetaD/2D_case
cp ../../../inputs/plumed_2.dat . 
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [23]:
grompp_lambda_2 = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../lambda_MetaD/2D_case/expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': '../lambda_MetaD/2D_case/sys2.top'},
                                     output_files={'-o': '../lambda_MetaD/2D_case/sys2_lambda.tpr'})
grompp_lambda_2.run()
gmx_output(grompp_lambda_2)

The process was executed successfully.


In [24]:
%%bash 
cd lambda_MetaD/2D_case 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

lambda_MetaD = gmx.commandline_operation('gmx',
                              arguments=['mdrun', '-plumed', '../plumed_2.dat'],
                              input_files={'-s': '../sys2_lambda.tpr'},
                              output_files={
                                  '-c': '../sys2_lambda.gro',
                                  '-e': '../lambda.edr',
                                  '-g': '../lambda.log',
                                  '-o': '../lambda.trr'})
lambda_MetaD.run()

if lambda_MetaD.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {lambda_MetaD.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')"

echo "${code}" >> lambda_MetaD.py

python lambda_MetaD.py

The process was executed successfully.
